In [67]:
# !  pip install wandb


In [75]:
import os
import sys
sys.path.append('../../src')
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
from datetime import datetime
import time

# import config

import data_process.neg_sample as ng_sample
# import evaluate_entity
from evaluate_entity import CustomHR, CustomNDCG, CustomRoc, CustomRoctop, CustomRecall_top, CustomPrecision_top
from model_entity import EntityCat
from data_utils import CatData
from utils.constants import DEFAULT_USER_COL,DEFAULT_ITEM_COL,DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL

from sklearn import metrics, preprocessing
# from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, recall_score, precision_score

import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

 
import re
from sentence_transformers import SentenceTransformer, util
from collections import Counter 

# import argparse
torch.manual_seed(0)

In [3]:
from ignite.engine import Engine, Events, create_supervised_trainer, create_supervised_evaluator, RemovableEventHandle
from ignite.metrics import Accuracy, Loss, Metric
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.exceptions import NotComputableError
from ignite.metrics.metric import sync_all_reduce, reinit__is_reduced
from ignite.contrib.handlers.tqdm_logger import ProgressBar
# from ignite.contrib.handlers import TensorboardLogger 
from ignite.contrib.handlers.wandb_logger import *

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
device

device(type='cpu')

In [6]:
device='cpu'
if device =='cpu':
    BATCH_SIZE = 20
else:
    BATCH_SIZE = 2**20
EPOCHS  = 20
TOP_K = 10
NEG_TEST = 100
NEG_TRAIN = 4

In [7]:
if device == 'cpu':
    use_amp=False
    df_train1  = ng_sample.read_feather("../../jobs/leave_one_train.csv")
    df_train2 = pd.read_csv("../../jobs/leave_one_train_neg.csv",encoding = "utf-8")
    df_test_ori = pd.read_csv("../../jobs/test_pos_neg.csv").iloc[:202,]
    df_all_features = pd.read_csv('../../jobs/merged_sub_clean.csv')
    df_train1 = df_train1.sort_values(by=[DEFAULT_USER_COL]).iloc[:100,].reset_index(drop=True)
    df_train2 = df_train2.sort_values(by=[DEFAULT_USER_COL]).iloc[:100*NEG_TRAIN,].reset_index(drop=True)
else:
    use_amp=True
    df_train1  = ng_sample.read_feather("../../jobs/leave_one_train.csv")
    df_train2 = pd.read_csv("../../jobs/leave_one_train_neg.csv")
    df_test_ori = pd.read_csv("../../jobs/test_pos_neg.csv")
    df_all_features = pd.read_csv('../../jobs/merged_sub_clean.csv')
    df_train1 = df_train1.sort_values(by=[DEFAULT_USER_COL]).reset_index(drop=True)
    df_train2 = df_train2.sort_values(by=[DEFAULT_USER_COL]).reset_index(drop=True)

In [8]:
df_train1[DEFAULT_RATING_COL] = 1

In [9]:
df_train1.head(3)

,userid,itemid,rating
0,7,309823,1
1,9,809208,1
2,9,136489,1


In [10]:
df_train2.head(3)

,userid,itemid,rating
0,7,516130,0
1,7,544280,0
2,7,372893,0


In [14]:
df_test_ori.head(7)

,userid,itemid,rating
0,7,703889,1
1,7,516136,0
2,7,544291,0
3,7,372900,0
4,7,954885,0
5,7,521338,0
6,7,532494,0


In [15]:
df_all_features.head(3)

,userid,timestamp,itemid,WindowID_user,Split,City,State,Country,Zip_user,DegreeType,...,ManagedOthers,ManagedHowMany,WindowID_job,City_job,State_job,Country_job,Zip_job,StartDate,EndDate,rating
0,7,2012-04-22 08:14:26.443,309823,3,Train,Roanoke,VA,US,24012,High School,...,No,0,3.0,Roanoke,VA,US,00nan,2012-04-20 19:29:22.367,2012-05-19 23:59:59,1
1,7,2012-04-28 12:17:58.630,703889,3,Train,Roanoke,VA,US,24012,High School,...,No,0,3.0,Roanoke,VA,US,00nan,2012-04-04 04:39:35.420,2012-05-03 23:59:59,1
2,106,2012-04-27 11:41:22.320,473209,3,Train,Nashville,TN,US,37207,Bachelor's,...,No,0,3.0,Nashville,TN,US,37210,2012-04-18 11:38:32.617,2012-05-17 23:59:59,1


In [17]:
def concat_index(df1, df2, NEG_TRAIN):
    df2.index = df2.index//NEG_TRAIN
    return pd.concat([df1, df2], axis=0).sort_index(kind='mregesort').reset_index(drop=True)

In [19]:
df_train_all = concat_index(df_train1, df_train2, NEG_TRAIN)
df_train_all

,userid,itemid,rating
0,7,309823,1
1,7,516130,0
2,7,544280,0
3,7,372893,0
4,7,954871,0
...,...,...,...
495,705,585072,1
496,705,45668,1
497,705,614923,1
498,705,277599,1


In [20]:
df_train_all['flag'] = 1
df_test_ori['flag'] = 0
df_all = pd.concat([df_train_all, df_test_ori], axis=0).reset_index(drop=True)

user features: 
       'WindowID_user', 'Split', 'City',
       'State', 'Country', 'Zip_user', 'DegreeType', 'Major', 'GraduationDate',
       'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed',
       'ManagedOthers', 'ManagedHowMany',
       
job features: 
       'WindowID_job', 'City_job',
       'State_job', 'Country_job', 'Zip_job', 'StartDate', 'EndDate',

### Choose the features for the model

In [28]:
user_features = ['City']
user_features_extend = [DEFAULT_USER_COL] + user_features

item_features = ['City_job']
item_features_extend =[DEFAULT_ITEM_COL] + item_features

base_features = [DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL, ]

In [29]:
def unique_filter(df_data, name_col):
    df_uni=df_data[~(df_data[name_col].duplicated())].reset_index(drop=True)
    return df_uni

In [30]:
def feature_merge(list_f, df_all, df_all_f, default_x_col):
    df_x_features = df_all_f[list_f]
    df_x_unique = unique_filter(df_x_features, default_x_col)
    df_merge_x = df_all.merge(df_x_unique, how='left', on=[default_x_col])
    return df_merge_x

In [31]:
def user_item_merge(mode, df_all, df_all_features, feature_list):
    if mode =='user':
        df_merge_x = feature_merge(feature_list, df_all, df_all_features,\
                                      DEFAULT_USER_COL)
    if mode == 'item':
        df_merge_x = feature_merge(feature_list, df_all, df_all_features, \
                                   DEFAULT_ITEM_COL)

    return df_merge_x

In [32]:
def mix_merge(df_all, df_all_features, f_list_user, f_list_item):
    df_merge_user = feature_merge(f_list_user, df_all, df_all_features, \
                                  DEFAULT_USER_COL)
    df_merge_item = feature_merge(f_list_item, df_all, df_all_features, \
                                  DEFAULT_ITEM_COL)
    df_merge_x = pd.concat([df_merge_user[f_list_user], \
                             df_merge_item[f_list_item]], axis=1)
    df_merge_x[DEFAULT_RATING_COL] = df_all[DEFAULT_RATING_COL]
    assert df_all.shape[0] == df_merge_x.shape[0], "wrong merge"
    return df_merge_x

In [33]:
df_mix_merge = mix_merge(df_all, df_all_features, user_features_extend, item_features_extend)

In [34]:
df_mix_merge.head()

,userid,City,itemid,City_job,rating
0,7,Roanoke,309823,Roanoke,1
1,7,Roanoke,516130,Albany,0
2,7,Roanoke,544280,Dearborn,0
3,7,Roanoke,372893,Houston,0
4,7,Roanoke,954871,Decatur,0


In [35]:
le = preprocessing.LabelEncoder()

In [36]:
le

LabelEncoder()

In [37]:
def cat_encode(df_data, list_f):
    for f in list_f:
        df_data[f] = le.fit_transform(df_data[f].astype('category').cat.codes.values)
    return df_data

In [39]:
features_to_code = df_mix_merge.columns
features_to_code

Index(['userid', 'City', 'itemid', 'City_job', 'rating'], dtype='object')

In [40]:
df_all_encode = cat_encode(df_mix_merge, features_to_code)
df_all_encode

,userid,City,itemid,City_job,rating
0,0,26,93,167,1
1,0,26,164,1,0
2,0,26,176,52,0
3,0,26,114,92,0
4,0,26,353,53,0
...,...,...,...,...,...
697,1,14,378,118,0
698,1,14,5,59,0
699,1,14,289,229,0
700,1,14,285,5,0


In [41]:
# df_all_encode[DEFAULT_RATING_COL] = df_all[DEFAULT_RATING_COL]

In [48]:
df_train = df_all_encode[df_all.flag==1]
df_test = df_all_encode[df_all.flag==0]

# df_train=df_train.drop(['flag'], axis=1)
# df_test=df_test.drop(['flag'], axis=1)

In [49]:
features_to_train = [DEFAULT_USER_COL, DEFAULT_ITEM_COL]+ user_features + item_features +[DEFAULT_RATING_COL]
df_train = df_train[features_to_train]
df_test = df_test[features_to_train]

In [50]:
log_name = "-".join(user_features)+'-'+'-'.join(item_features)
log_name

'City-City_job'

In [51]:
df_train=df_train[features_to_train]
df_test=df_test[features_to_train]

num_feature=[]
features_to_train.remove(DEFAULT_RATING_COL)

In [52]:
def data_split_user(df_train, val_size=0.2):
    unique_user = df_train[DEFAULT_USER_COL].unique()
    val_user = np.random.choice(unique_user, int(val_size*len(unique_user)), replace=False)
    df_train_split = df_train[~(df_train[DEFAULT_USER_COL].isin(val_user))]
    df_val_split = df_train[df_train[DEFAULT_USER_COL].isin(val_user)]
    return df_train_split, df_val_split

In [53]:
df_train_split, df_val_split = data_split_user(df_train, val_size=0.2)

In [54]:
np_train = df_train_split.values
np_val = df_val_split.values
np_test = df_test.values

In [55]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [56]:
train_dataset = CatData(np_train)
val_dataset = CatData(np_val)
test_dataset = CatData(np_test) 
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0,  worker_init_fn=seed_worker,generator=g)
val_loader = data.DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False, num_workers=0,  worker_init_fn=seed_worker,generator=g)
test_loader = data.DataLoader(test_dataset, batch_size=NEG_TEST+1, shuffle=False, num_workers=0,worker_init_fn=seed_worker,generator=g )

In [57]:
embedding_size = []
for c in features_to_train:
    num_unique_values = int(df_all_encode[c].nunique())
    embed_dim = int(min(np.ceil(num_unique_values/2), 50))
    embedding_size.append([num_unique_values, embed_dim])  

# job title embedding

In [78]:
jobs = pd.read_csv('../../tsv/jobs.tsv', sep='\t', encoding='utf-8', on_bad_lines='skip')


/var/folders/dn/91bgkt1972sb4mq69k19zp840000gp/T/ipykernel_40566/4047767119.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  jobs = pd.read_csv('../../tsv/jobs.tsv', sep='\t', encoding='utf-8', on_bad_lines='skip')


In [79]:
jobs.rename(columns = {'JobID':"item_id", 'City':'City_job', \
                       'State':'State_job', 'Country':'Country_job', \
                      'Zip5':'Zip_job'}, inplace=True)

In [80]:
jobs.head()

,item_id,WindowID,Title,Description,Requirements,City_job,State_job,Country_job,Zip_job,StartDate,EndDate
0,1,1,Security Engineer/Technical Lead,<p>Security Clearance Required:&nbsp; Top Secr...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,Washington,DC,US,20531,2012-03-07 13:17:01.643,2012-04-06 23:59:59
1,4,1,SAP Business Analyst / WM,<strong>NO Corp. to Corp resumes&nbsp;are bein...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,Charlotte,NC,US,28217,2012-03-21 02:03:44.137,2012-04-20 23:59:59
2,7,1,P/T HUMAN RESOURCES ASSISTANT,<b> <b> P/T HUMAN RESOURCES ASSISTANT</b> <...,Please refer to the Job Description to view th...,Winter Park,FL,US,32792,2012-03-02 16:36:55.447,2012-04-01 23:59:59
3,8,1,Route Delivery Drivers,CITY BEVERAGES Come to work for the best in th...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:10.077,2012-04-02 23:59:59
4,9,1,Housekeeping,I make sure every part of their day is magica...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:11.88,2012-04-02 23:59:59


In [81]:
jobs['Title'] = jobs['Title'].fillna('')
jobs['Description'] = jobs['Description'].fillna('')
jobs['Requirements'] = jobs['Requirements'].fillna('')

#jobs_US_base_line['Requirements'] = jobs_US_base_line['Requirements'].fillna('')
sep = " "

def preprocessor(text):
    text = text.replace('\\r', '').replace('&nbsp', '').replace('\n', '')
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

jobs['Title'] = jobs['Title'].astype(dtype='str').apply(preprocessor)
jobs['Description'] = jobs['Description'].astype(dtype='str').apply(preprocessor)
jobs['Requirements'] = jobs['Requirements'].astype(dtype='str').apply(preprocessor)


In [82]:
jobs.loc[:,"job_text"] = jobs['Title'] +sep+ jobs['Description'] +sep+jobs['Requirements']


In [83]:
jobs.columns

Index(['item_id', 'WindowID', 'Title', 'Description', 'Requirements',
       'City_job', 'State_job', 'Country_job', 'Zip_job', 'StartDate',
       'EndDate', 'job_text'],
      dtype='object')

In [85]:
jobs[['Title', 'Description', 'Requirements','job_text']].head(3)

,Title,Description,Requirements,job_text
0,security engineer technical lead,security clearance required top secret job num...,skill set network security tools webdefend web...,security engineer technical lead security clea...
1,sap business analyst wm,no corp to corp resumes are being considered f...,what you need four year college degreeminimum ...,sap business analyst wm no corp to corp resume...
2,p t human resources assistant,p t human resources assistant 1 2 years exper...,please refer to the job description to view th...,p t human resources assistant p t human resou...


In [87]:
sbert_model =  SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [88]:

jobs["Title_em"]=  jobs['Title'].apply(lambda x : sbert_model.encode(x))

KeyboardInterrupt: 

In [ ]:
jobs['Title'] = jobs['Title'].fillna('')
 

def preprocessor(text):
    text = text.replace('\\r', '').replace('&nbsp', '').replace('\n', '')
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

jobs['Title'] = jobs['Title'].astype(dtype='str').apply(preprocessor)
 
sbert_model =  SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

jobs["Title_em"]=  jobs['Title'].apply(lambda x : sbert_model.encode(x))

In [58]:
model = EntityCat(embedding_size = embedding_size, num_numerical_cols = len(num_feature),
               output_size = 2)
model.to(device)

EntityCat(
  (all_embeddings): ModuleList(
    (0): Embedding(36, 18)
    (1): Embedding(413, 50)
    (2): Embedding(35, 18)
    (3): Embedding(230, 50)
  )
  (embedding_dropout): Dropout(p=0.4, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=136, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=100, out_features=2, bias=True)
  )
)

In [59]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [60]:
print(model)

EntityCat(
  (all_embeddings): ModuleList(
    (0): Embedding(36, 18)
    (1): Embedding(413, 50)
    (2): Embedding(35, 18)
    (3): Embedding(230, 50)
  )
  (embedding_dropout): Dropout(p=0.4, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=136, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=100, out_features=2, bias=True)
  )
)


In [61]:
def output_trans_loss(output):
    return output['y_pred'], output['label']

val_metrics_train = {
    'auc': CustomRoc(),
    "loss": Loss(criterion, output_transform=output_trans_loss)
}

val_metrics_test = {
    'hr': CustomHR(),
    'ndcg': CustomNDCG(),
    'auc': CustomRoc(),
    'roc_top': CustomRoctop(),
    'recall_top': CustomRecall_top(threshold=0.5),
    'precision_top': CustomPrecision_top(threshold=0.5),
    "loss": Loss(criterion, output_transform=output_trans_loss)
}

In [62]:
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch[0].to(device), batch[1].to(device)
    with torch.cuda.amp.autocast(enabled=use_amp):
        y_pred = model(x)[:,1]
        loss = criterion(y_pred, y.float())
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    return loss.item()

def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, label = batch[0].to(device), batch[1].to(device)
        y_pred = model(x)[:,1]
        label=label.float()
        return {'label':label, 'y_pred':y_pred}

def test_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, label = batch[0].to(device), batch[1].to(device)
        y_pred = model(x)[:,1]
        label=label.float()
        y_pred_top, indices = torch.topk(y_pred, engine.state.topk)
        y_pred_top = y_pred_top.detach().cpu().numpy()
        reco_item = torch.take(x[:,1], indices).cpu().numpy().tolist()
        pos_item = x[0,1].cpu().numpy().tolist()  # ground truth, item id
        label_top = label[indices].cpu().numpy()
        return {'pos_item':pos_item, 'reco_item':reco_item, 'y_pred_top':y_pred_top, 'label_top':label_top, 'label':label, 'y_pred':y_pred}

trainer = Engine(train_step)

train_evaluator = Engine(validation_step)
# train_evaluator.state_dict_user_keys.append('topk')

val_evaluator = Engine(validation_step)
# val_evaluator.state_dict_user_keys.append('topk')

test_evaluator = Engine(test_step)
test_evaluator.state_dict_user_keys.append('topk')

# @val_evaluator.on(Events.STARTED)
# def init_user_value():
#     val_evaluator.state.topk=3
    
# @train_evaluator.on(Events.STARTED)
# def init_user_value():
#     train_evaluator.state.topk=3

@train_evaluator.on(Events.STARTED)
def init_user_value():
    test_evaluator.state.topk=20
    
    
# Attach metrics to the evaluators
for name, metric in val_metrics_train.items():
    metric.attach(train_evaluator, name)

for name, metric in val_metrics_train.items():
    metric.attach(val_evaluator, name)

    
for name, metric in val_metrics_test.items():
    metric.attach(test_evaluator, name)
    
# Eearly_stop 
def score_function(engine):
    val_loss = engine.state.metrics['auc']
    return val_loss

Eearly_stop_handler = EarlyStopping(patience=10, score_function=score_function, trainer=trainer)
val_evaluator.add_event_handler(Events.COMPLETED, Eearly_stop_handler)

In [73]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    train_evaluator.run(train_loader)
    metrics = train_evaluator.state.metrics
    auc = metrics['auc']
    loss = metrics['loss']
    print(f'Training Results- Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
          Avg auc:{auc:.2f}')


@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    val_evaluator.run(val_loader)
    metrics = val_evaluator.state.metrics
    auc = metrics['auc']
    loss = metrics['loss']
    print(f'Validation Results- Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
          Avg auc:{auc:.2f}')

    
@trainer.on(Events.COMPLETED)
def log_test_results(trainer):
    test_evaluator.run(test_loader)
    metrics = test_evaluator.state.metrics
    hr = metrics['hr']
    ndcg = metrics['ndcg']
    auc = metrics['auc']
    roc_top = metrics['roc_top']
    recall = metrics['recall_top']
    precision = metrics['precision_top']
    loss = metrics['loss']
    print(f"Test Results - Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
     Avg ndcg: {ndcg:.2f}  Avg auc: {auc:.2f}  Avg auc_top: {roc_top:.2f} \
      Avg recall: {recall:.2f}  Avg precision: {precision:.2f}")

pbar = ProgressBar(persist=False)
# pbar.attach(trainer)
pbar.attach(trainer)
# from ignite.handlers import Timer
# timer = Timer(average=True)
# timer.attach(trainer,
#              start=Events.STARTED,
#              resume=Events.EPOCH_STARTED,
#              pause=Events.EPOCH_COMPLETED,
#              step=Events.EPOCH_COMPLETED)

In [74]:
trainer.run(train_loader, max_epochs=20)

/Users/che/miniforge3/envs/torch/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[1/20]   5%|5          [00:00<?]

[1/20]   5%|5          [00:00<?]

[1/20]   5%|5          [00:00<?]

[1/20]   5%|5          [00:00<?]

2022-06-30 09:58:21,745 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training
2022-06-30 09:58:21,759 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training
2022-06-30 09:58:21,772 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training
2022-06-30 09:58:21,786 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


Training Results- Epoch[1]  Avg loss: 0.65           Avg auc:0.81
Validation Results- Epoch[1]  Avg loss: 0.70           Avg auc:0.78
Training Results- Epoch[1]  Avg loss: 0.65           Avg auc:0.81
Validation Results- Epoch[1]  Avg loss: 0.70           Avg auc:0.78
Training Results- Epoch[1]  Avg loss: 0.65           Avg auc:0.81
Validation Results- Epoch[1]  Avg loss: 0.70           Avg auc:0.78
Training Results- Epoch[1]  Avg loss: 0.65           Avg auc:0.81
Validation Results- Epoch[1]  Avg loss: 0.70           Avg auc:0.78
Test Results - Epoch[1]  Avg loss: 0.78      Avg ndcg: 0.14  Avg auc: 0.54  Avg auc_top: 0.21       Avg recall: 0.50  Avg precision: 0.04
Test Results - Epoch[1]  Avg loss: 0.78      Avg ndcg: 0.14  Avg auc: 0.54  Avg auc_top: 0.21       Avg recall: 0.50  Avg precision: 0.04
Test Results - Epoch[1]  Avg loss: 0.78      Avg ndcg: 0.14  Avg auc: 0.54  Avg auc_top: 0.21       Avg recall: 0.50  Avg precision: 0.04
Test Results - Epoch[1]  Avg loss: 0.78      Avg n

State:
	iteration: 20
	epoch: 1
	epoch_length: 20
	max_epochs: 20
	output: 0.5318541526794434
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

In [69]:
# wandb_logger = WandBLogger(
#     project="pytorch-jrs",
#     name=log_name,
#     config={"max_epochs": EPOCHS,"batch_size":BATCH_SIZE, 'Freatures':user_features+item_features},
#     tags=["entity", "jrs"]
# )

# to_save = {'model': model}
# checkpoint_handler = ModelCheckpoint(
#     wandb_logger.run.dir,
#     n_saved=2, filename_prefix='best',
#     score_name="auc",
#     global_step_transform=global_step_from_engine(trainer)
# )

# val_evaluator.add_event_handler(Events.COMPLETED, checkpoint_handler, to_save)

    
# wandb_logger.attach_output_handler(
#     trainer,
#     event_name=Events.ITERATION_COMPLETED,
#     tag="training",
#     output_transform=lambda loss: {"loss": loss}
# )

# wandb_logger.attach_output_handler(
#     train_evaluator,
#     event_name=Events.EPOCH_COMPLETED,
#     tag="training",
#     metric_names=['loss','auc'],
#     global_step_transform=lambda *_: trainer.state.iteration,
# )

# wandb_logger.attach_output_handler(
#     val_evaluator,
#     event_name=Events.EPOCH_COMPLETED,
#     tag="validation",
#     metric_names=['loss',"auc"],
#     global_step_transform=lambda *_: trainer.state.iteration,
# )


# wandb_logger.attach_output_handler(
#     test_evaluator,
#     event_name=Events.COMPLETED,
#     tag="test",
#     metric_names=['loss',"auc", 'hr', 'ndcg', 'roc_top', 'recall_top', 'precision_top'],
#     global_step_transform=lambda *_: trainer.state.iteration,
# )


# wandb_logger.attach_opt_params_handler(
#     trainer,
#     event_name=Events.ITERATION_STARTED,
#     optimizer=optimizer,
#     param_name='lr'  # optional
# )

# wandb_logger.watch(model) 

# trainer.run(train_loader, max_epochs=EPOCHS)
# wandb_logger.close()